In [2]:
pip install pennylane-sf pennylane-qiskit pennylane-cirq 

     |████████████████████████████████| 631 kB 49 kB/s s eta 0:00:01
     |████████████████████████████████| 409 kB 2.3 kB/s  eta 0:00:01
     |████████████████████████████████| 47 kB 24 kB/s s eta 0:00:01
     |████████████████████████████████| 55 kB 25 kB/s s eta 0:00:01
     |████████████████████████████████| 328 kB 4.1 kB/s  eta 0:00:01
     |████████████████████████████████| 1.5 MB 48 kB/s s eta 0:00:01    |▍                               | 20 kB 32.2 MB/s eta 0:00:01     |█████████████████████████████▋  | 1.4 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 11 kB/s  eta 0:00:011
     |████████████████████████████████| 1.3 MB 78 kB/s s eta 0:00:01��███████████████▉        | 952 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 46 kB/s s eta 0:00:01
     |████████████████████████████████| 52 kB 8.4 kB/s  eta 0:00:01
     |████████████████████████████████| 13.7 MB 6.3 kB/s eta 0:00:01�████████████████████▍      | 10.9 MB 2.9 MB/s eta 0:00:0

In [4]:
pip install nlopt

     |████████████████████████████████| 420 kB 7.4 kB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pennylane as qml
from pennylane import numpy as np
import nlopt 
import random
import qiskit


import warnings
#warnings.filterwarnings('ignore')

In [6]:
dev = qml.device("strawberryfields.fock", wires=8, cutoff_dim=4)
#quantum optical device

In [7]:
def layer(theta, phi, wires):
    M = len(wires)
    phi_nonlinear = np.pi / 2

    qml.templates.Interferometer(
        theta, phi, np.zeros(M), wires=wires, mesh="triangular",
    )

    for i in wires:
        qml.Kerr(phi_nonlinear, wires=i)

In [8]:
@qml.qnode(dev)
def quantum_neural_net(var, x):
    wires = list(range(len(x)))

    # Encode input x into a sequence of quantum fock states
    for i in wires:
        qml.FockState(x[i], wires=i)

    # "layer" subcircuits
    for i, v in enumerate(var):
        layer(v[: len(v) // 2], v[len(v) // 2 :], wires)

    return [qml.expval(qml.NumberOperator(w)) for w in wires]

In [9]:
def square_loss(labels, predictions):
    term = 0
    for l, p in zip(labels, predictions):
        lnorm = l / np.linalg.norm(l)
        pnorm = p / np.linalg.norm(p)

        term = term + np.abs(np.dot(lnorm, pnorm.T)) ** 2

    return 1 - term / len(labels)

In [10]:
def cost(var, data_input, labels):
    predictions = np.array([quantum_neural_net(var, x) for x in data_input])
    sl = square_loss(labels, predictions)

    return sl

In [11]:
# Define the CNOT input-output states (dual-rail encoding) and initialize
# them as non-differentiable.

X = np.array([[1, 0, 1, 0, 1, 0, 1, 0],
              [1, 0, 0, 1, 1, 0, 0, 1],
              [0, 1, 1, 0, 0, 1, 1, 0],
              [0, 1, 1, 0, 0, 1, 1, 0]], requires_grad=False)

Y = np.array([[1, 0, 1, 0, 0, 1, 0, 1],
              [1, 0, 0, 1, 1, 0, 0, 1],
              [0, 1, 1, 0, 0, 1, 1, 0],
              [0, 1, 0, 1, 1, 0, 1, 0]], requires_grad=False)

In [12]:
num_layers = 2
M = len(X[0])
num_variables_per_layer = M * (M - 1)

var_init = (4 * np.random.rand(num_layers, num_variables_per_layer) - 2) * np.pi
print(var_init)

[[ 4.2825563  -6.1408911  -5.87373511  1.3332275   1.47428032  0.78687751
  -0.67547308  2.70547879 -6.1962518  -4.94895342 -1.04652099  1.71944016
  -2.13983595 -1.61190231 -4.30495892  0.85568442  2.72112333  1.10548071
  -5.79529182  4.66344621 -0.60153857  4.06944977  5.29618298  6.07811723
  -5.30767856  1.950211    4.47265064  2.99498513  2.41731782 -5.66215503
   0.25771968  0.17024795 -5.01563107 -0.30311527  3.66998756 -0.71130326
   0.99974458  5.2788874   3.05365741 -0.55396044 -2.242438   -3.01206251
   5.27666143 -3.50533227  0.32725223 -2.71499779  1.08558767 -4.94215991
   2.98221314 -3.72856634  1.40946809  1.55845192 -0.45166166  0.71372067
   2.32947992  1.81311356]
 [ 1.19085905  0.97988344 -1.52528925  3.17464111  4.02685542 -3.70803056
  -1.58524594  4.16561155 -5.63365316 -3.40950906  3.41168444  5.77394119
   3.67308117 -5.37457453  5.02401826 -1.88782095  3.46766627 -4.87657163
   4.35525705  2.88800124  4.25750585 -0.02907014  4.38571044  1.47574258
  -4.027379

In [ ]:
cost_grad = qml.grad(cost)

print_every = 1

# Wrap the cost so that NLopt can use it for gradient-based optimizations
evals = 0
def cost_wrapper(var, grad=[]):
    global evals
    evals += 1

    if grad.size > 0:
        # Get the gradient for `var` by first "unflattening" it
        var_grad = cost_grad(var.reshape((num_layers, num_variables_per_layer)), X, Y)
        grad[:] = var_grad.flatten()
    cost_val = cost(var.reshape((num_layers, num_variables_per_layer)), X, Y)

    if evals % print_every == 0:
        print(f"Iter: {evals:4d}    Cost: {cost_val:.4e}")

    return float(cost_val)


# Choose an algorithm
opt_algorithm = nlopt.LD_LBFGS  # Gradient-based
# opt_algorithm = nlopt.LN_BOBYQA  # Gradient-free

opt = nlopt.opt(opt_algorithm, num_layers*num_variables_per_layer)

opt.set_min_objective(cost_wrapper)

opt.set_lower_bounds(-2*np.pi * np.ones(num_layers*num_variables_per_layer))
opt.set_upper_bounds(2*np.pi * np.ones(num_layers*num_variables_per_layer))

var = opt.optimize(var_init.flatten())
var = var.reshape(var_init.shape)

In [ ]:
print(f"The optimized parameters (layers, parameters):\n {var}\n")

Y_pred = np.array([quantum_neural_net(var, x) for x in X])
for i, x in enumerate(X):
    print(f"{x} --> {Y_pred[i].round(2)}, should be {Y[i]}")

In [ ]:
quantum_neural_net(var_init, X[0])

In [ ]:
print(quantum_neural_net.draw())